In [ ]:
import pandas as pd
import requests as rq
from bs4 import BeautifulSoup
import os
import json

In [ ]:
response = rq.get("https://sammlungen.ub.uni-frankfurt.de/kolonialbibliothek/periodical/titleinfo/7720783")

In [ ]:
response

In [ ]:
response.text

In [ ]:
df_jahrgänge = pd.read_html(response.content, attrs = {"id": "titleInfoJournal_volumes"}, extract_links= "all")[0]
df_jahrgänge

In [ ]:
df_jahrgänge[0].values[3]

In [ ]:
soup = BeautifulSoup(response.text, "html.parser")
table = soup.find("table", id = "titleInfoJournal_volumes")
table

In [ ]:
links = []
for row in table.find_all("tr")[1:]:
    link_tag = row.find("a", class_="childLink")
    if link_tag and link_tag.has_attr("href"):
        links.append(link_tag["href"])
    else:
        links.append(None)

list_ids = [link.split("/")[-1] for link in links]
unique_ids = sorted(list(set(list_ids)))

In [ ]:
"https://sammlungen.ub.uni-frankfurt.de/i3f/v20/" + unique_ids[0] + "/manifest"

In [ ]:
manifest_urls = ["https://sammlungen.ub.uni-frankfurt.de/i3f/v20/" + id + "/manifest" for id in unique_ids]
manifest_urls

In [ ]:
for link in manifest_urls:
    response_iiif = rq.get(link)
    data = response_iiif.json()

    filename = os.path.join("output", (data["label"] + ".json").replace(" ", "_").replace(":", ""))
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=4)
